In [1]:
from IPython.display import display

from io import StringIO
import itertools
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd

from mosek.fusion import Model, Domain, Expr, ObjectiveSense, SolutionStatus


In [2]:
# supply = pd.read_csv('inputs/stochastic_example_offer.csv').set_index('id')
supply = pd.read_csv('inputs/20201007_da_co.classified.csv').set_index('id')
nsupply = len(supply)
print(supply.shape)
supply.head()

(2851, 4)


,node,capacity (MW),offer ($/MW),type
id,,,,
A3153_0,North,1.0,8.16,inflexible
A3153_1,North,245.0,9.54,inflexible
A3153_2,North,1.0,14.27,inflexible
A3153_3,North,72.0,14.28,inflexible
A3167_0,Central,330.0,8.74,inflexible


In [3]:
# demand = pd.read_csv('inputs/stochastic_example_bid.csv').set_index('id')
demand = pd.read_csv('inputs/20201007_bids_cb.processed.csv').set_index('id')
ndemand = len(demand)
ushed = demand['bid ($/MW)'].max()
print(demand.shape)
demand.head()

(309, 3)


,node,demand (MW),bid ($/MW)
id,,,
122073561,North,6.0,2000.0
767746013,North,14.0,2000.0
122073033,Central,1228.0,2000.0
122073042,North,1644.0,2000.0
450523929,Central,132.4,2000.0


In [4]:
# scenarios = pd.read_csv('inputs/stochastic_example_scenarios.csv').set_index('scenario')
scenarios = pd.read_csv('inputs/20201007_da_co.scenarios.csv').set_index('scenario')
scenarios = scenarios.loc[100:,:]
nscenarios = len(scenarios.index.unique())
pi_w = 1. / nscenarios

# scenarios = scenarios.loc[[0]]
# nscenarios = 1
# pi_w = 1.

# scenarios = scenarios.loc[[]]
# nscenarios = 0
# pi_w = 0.

nscenarios = len(scenarios)
print(scenarios.shape)
scenarios.head()

(236700, 2)


,id,actual (MW)
scenario,,
100,A3200,57.865122
100,A3205,0.625427
100,A3213,12.398359
100,A3215,57.733552
100,A3375,58.539169


In [5]:
nodes = sorted(set(supply['node']) | set(demand['node']))
nnodes = len(nodes)
print(nodes)

['Central', 'North', 'South']


In [6]:
lines = pd.DataFrame([
    ['%s-%s' % (src, dest), src, dest, 5e3, 1e4] 
    for src, dest in itertools.combinations(nodes, 2)
], columns=['id', 'source', 'dest', 'capacity (MW)', 'susceptance (S)']).set_index('id')
nlines = len(lines)
lines

,source,dest,capacity (MW),susceptance (S)
id,,,,
Central-North,Central,North,5000.0,10000.0
Central-South,Central,South,5000.0,10000.0
North-South,North,South,5000.0,10000.0


In [7]:
wind_supply = pd.merge(supply.query('type == "wind"'), scenarios.loc[scenarios.iloc[0].name], 
                       left_index=True, right_on='id', how='outer')
by_node = pd.DataFrame({
    'total demand': demand.groupby('node')['demand (MW)'].sum(),
    'total capacity': supply.groupby('node')['capacity (MW)'].sum(),
    'wind capacity': wind_supply.groupby('node')['capacity (MW)'].sum(),
    'wind actual': wind_supply.groupby('node')['actual (MW)'].sum(),
}).fillna(0.)
by_node.append(by_node.sum().rename('Total'))

,total demand,total capacity,wind capacity,wind actual
node,,,,
Central,28181.2,66463.8,13666.6,4319.131984
North,12854.2,43894.8,24364.4,7054.930145
South,16425.5,35474.6,45.0,13.979324
Total,57460.9,145833.2,38076.0,11388.041453


In [8]:
training_supply = pd.read_csv('output/training_supply.csv').set_index('id')
training_demand = pd.read_csv('output/training_demand.csv').set_index('id')

In [9]:
M = Model('power')

log = StringIO()
M.setLogHandler(log)
theta = M.variable('theta', nnodes, Domain.inRange(
    [0.] + [-np.pi] * (nnodes - 1), 
    [0.] + [np.pi] * (nnodes - 1), 
))
pD = M.variable('pD', ndemand, Domain.equalsTo(training_demand['DA'].values))
pG = M.variable('pG', nsupply, Domain.equalsTo(training_supply['DA'].values))
UD = demand['bid ($/MW)'].values
CG = supply['offer ($/MW)'].values

line_constraints = Domain.inRange(-lines['capacity (MW)'].values, 
                                  lines['capacity (MW)'].values)
line_flows = M.variable('flows', nlines, line_constraints)
src_idx = [nodes.index(x) for x in lines.source]
dst_idx = [nodes.index(x) for x in lines.dest]
phase_diff = Expr.sub(theta.pick(src_idx), theta.pick(dst_idx))
phase_flow = M.constraint(Expr.sub(line_flows, 
                                  Expr.mulElm(lines['susceptance (S)'].values, phase_diff)),
                         Domain.equalsTo(0.))
node_balance_eqs = []
for node in nodes:
    supply_idx = np.flatnonzero(supply.node == node).astype('int32')
    demand_idx = np.flatnonzero(demand.node == node).astype('int32')
    node_supply = Expr.sum(pG.pick(supply_idx))
    node_demand = Expr.sum(pD.pick(demand_idx))
    balance = Expr.sub(node_demand, node_supply)
    direction = [1 if line.source == node else 
                 -1 if line.dest == node else 
                 0 for line_idx, line in lines.iterrows()]
    balance = Expr.add(balance, Expr.dot(direction, line_flows))
    balance_eq = M.constraint(node + 'balance', balance, Domain.equalsTo(0.))
    node_balance_eqs.append(balance_eq)

obj = Expr.sub(Expr.dot(UD, pD), Expr.dot(CG, pG))

M.constraint(pG.index(1), Domain.equalsTo(10.))

inflexible_idx = np.flatnonzero(supply.type == 'inflexible').astype('int32')
gdown = -supply['capacity (MW)'].values.copy()
gdown[inflexible_idx] = 0.
gup = supply['capacity (MW)'].values.copy()
gup[inflexible_idx] = 0.

wind_idx = np.flatnonzero(supply.type == 'wind').astype('int32')
nwind = len(wind_idx)
# flexible_idx = np.flatnonzero(supply.type == 'flexible')

node_balance_eqs_rt = []
for w_id, w in scenarios.groupby(level='scenario'):
    assert list(w.id) == list(supply.iloc[wind_idx].index)
    wind_actual = w['actual (MW)'].values
    
    theta_w = M.variable('theta%s' % w_id, nnodes, Domain.inRange(
        [0.] + [-np.pi] * (nnodes - 1), 
        [0.] + [np.pi] * (nnodes - 1), 
    ))
    pDrt = M.variable('pDrt%s' % w_id, ndemand, Domain.lessThan(0.))
    pGrt = M.variable('pGrt%s' % w_id, nsupply, Domain.inRange(gdown, gup))
    pGspill = M.variable('pGspill%s' % w_id, nwind, Domain.inRange(0., wind_actual))

    M.constraint(Expr.add(pD, pDrt), Domain.inRange(0., demand['demand (MW)'].values))
    M.constraint(Expr.add(pG, pGrt), Domain.inRange(0., supply['capacity (MW)'].values))
    
    total_wind = Expr.add(pG.pick(wind_idx), pGrt.pick(wind_idx))
    total_wind = Expr.add(total_wind, pGspill)
    M.constraint(total_wind, Domain.equalsTo(wind_actual))

    line_flows_w = M.variable('flows%s' % w_id, nlines, line_constraints)
    phase_diff_w = Expr.sub(theta_w.pick(src_idx), theta_w.pick(dst_idx))
    phase_flow_w = M.constraint(Expr.sub(line_flows_w, 
                                         Expr.mulElm(lines['susceptance (S)'].values, phase_diff_w)),
                                Domain.equalsTo(0.))
    node_balance_eqs_w = []
    for node in reversed(nodes):
        supply_idx = np.flatnonzero(supply.node == node).astype('int32')
        demand_idx = np.flatnonzero(demand.node == node).astype('int32')

        node_supply = Expr.sum(pG.pick(supply_idx))
        node_demand = Expr.sum(pD.pick(demand_idx))
        node_supply_w = Expr.sum(pGrt.pick(supply_idx))
        node_demand_w = Expr.sum(pDrt.pick(demand_idx))
        balance_w = Expr.sub(node_demand, node_supply)
        balance_w = Expr.add(balance_w, node_demand_w)
        balance_w = Expr.sub(balance_w, node_supply_w)
        direction = [1 if line.source == node else 
                     -1 if line.dest == node else 
                     0 for line_idx, line in lines.iterrows()]
        balance_w = Expr.add(balance_w, Expr.dot(direction, line_flows_w))
        balance_eq_w = M.constraint(node + 'balance%s' % w_id, balance_w, Domain.equalsTo(0.))
        node_balance_eqs_w.append(balance_eq_w)
    
    node_balance_eqs_rt.append(node_balance_eqs_w)
    
    obj_w = Expr.sub(Expr.mul(ushed, Expr.sum(pDrt)), Expr.dot(CG, pGrt))
    obj = Expr.add(obj, Expr.mul(pi_w, obj_w))

M.objective('obj', ObjectiveSense.Maximize, obj)
M.solve()

if M.getPrimalSolutionStatus() != SolutionStatus.Optimal:
    M.writeTask('proj1.opf')

In [10]:
demand_res = demand.copy()
demand_res['DA'] = pD.level()

supply_res = supply.copy()
supply_res['DA'] = pG.level()

lines_res = lines.copy()
lines_res['DA'] = line_flows.level()

prices = pd.DataFrame({
    'DA': [e.dual()[0] for e in node_balance_eqs],
}, index=nodes)

for i, w_id in enumerate(scenarios.index.unique()):
    demand_res['dRT%s' % w_id] = M.getVariable('pDrt%s' % w_id).level()
    demand_res['RT%s' % w_id] = demand_res['DA'] + demand_res['dRT%s' % w_id]
    supply_res['dRT%s' % w_id] = M.getVariable('pGrt%s' % w_id).level()
    supply_res['RT%s' % w_id] = pG.level() + M.getVariable('pGrt%s' % w_id).level()
    prices['RT%s' % w_id] = [e.dual()[0] for e in node_balance_eqs_rt[i]]
    lines_res['RT%s' % w_id] = M.getVariable('flows%s' % w_id).level()

In [11]:
with pd.option_context('display.float_format', '{:.1f}'.format):
    display(supply_res.reset_index().groupby(['node', 'type']).nth(0).iloc[:,0:10])

id  capacity (MW)  offer ($/MW)    DA  dRT100  RT100  \
node    type                                                                    
Central flexible    A3182_1            5.0          52.4   0.0     0.0    0.0   
        inflexible  A3167_0          330.0           8.7 330.0     0.0  330.0   
        wind          A3205            2.0           0.0   2.0    -1.4    0.6   
North   flexible    A3220_0            6.0          74.8   0.0     0.0    0.0   
        inflexible  A3153_0            1.0           8.2   1.0     0.0    1.0   
        wind          A3200          200.0           0.0   0.0    57.9   57.9   
South   flexible    A8558_0           55.0          41.6  55.0   -55.0    0.0   
        inflexible  A8541_0           50.0          21.4   0.0     0.0    0.0   
        wind        A8897_0           45.0           0.0  45.0   -31.0   14.0   

                    dRT101  RT101  dRT102  RT102  
node    type                                      
Central flexible       0.0    0.0     0.0    0.0  
        inflexible     0.0  330.0     0.0  330.0  
        wind          -2.0    0.0    -2.0    0.0  
North   flexible       0.0    0.0     0.0    0.0  
        inflexible     0.0    1.0     0.0    1.0  
        wind          71.0   71.0    73.7   73.7  
South   flexible     -55.0    0.0   -55.0    0.0  
        inflexible     0.0    0.0     0.0    0.0  
        wind         -29.4   15.6   -28.7   16.3

In [12]:
with pd.option_context('display.float_format', '{:.1f}'.format):
    display(supply_res.reset_index().groupby('node').sum().iloc[:,0:10])
supply_res.to_csv('output/test_supply.csv')

,capacity (MW),offer ($/MW),DA,dRT100,RT100,dRT101,RT101,dRT102,RT102,dRT103
node,,,,,,,,,,
Central,66463.8,46199.9,29028.9,-6289.2,22739.7,-7825.6,21203.3,-8095.4,20933.5,-5425.4
North,43894.8,42598.4,9144.0,7167.4,16311.4,8702.1,17846.1,8971.2,18115.2,6206.5
South,35474.6,13688.3,19149.5,-878.1,18271.4,-876.5,18273.0,-875.8,18273.7,-781.1


In [13]:
with pd.option_context('display.float_format', '{:.1f}'.format):
    display(demand_res.reset_index().groupby('node').nth(0).iloc[:,0:10])
demand_res.to_csv('output/test_demand.csv')

,id,demand (MW),bid ($/MW),DA,dRT100,RT100,dRT101,RT101,dRT102,RT102
node,,,,,,,,,,
Central,122073033,1228.0,2000.0,1228.0,-0.0,1228.0,-0.0,1228.0,-0.0,1228.0
North,122073561,6.0,2000.0,6.0,-0.0,6.0,-0.0,6.0,-0.0,6.0
South,1656071289,20.7,2000.0,20.7,-0.0,20.7,-0.0,20.7,-0.0,20.7


In [14]:
with pd.option_context('display.float_format', '{:.2f}'.format):
    display(prices.iloc[:,0:10])

,DA,RT100,RT101,RT102,RT103,RT104,RT105,RT106,RT107,RT108
Central,-0.00,0.04,0.00,0.00,0.05,0.04,0.05,0.04,0.00,0.04
North,-0.00,0.04,0.00,0.00,0.05,0.04,0.05,0.04,0.00,0.04
South,-0.00,0.04,0.00,0.00,0.05,0.04,0.05,0.04,0.00,0.04


In [15]:
with pd.option_context('display.float_format', '{:.0f}'.format):
    display(lines_res.iloc[:,0:10])

,source,dest,capacity (MW),susceptance (S),DA,RT100,RT101,RT102,RT103,RT104
id,,,,,,,,,,
Central-North,Central,North,5000,10000,1537,-2948,-3972,-4152,-2340,-2718
Central-South,Central,South,5000,10000,-636,-2440,-2952,-3042,-2184,-2325
North-South,North,South,5000,10000,-2173,509,1020,1109,156,393


In [16]:
da_obj = demand_res['DA'].dot(demand['bid ($/MW)']) - supply_res['DA'].dot(supply['offer ($/MW)'])
objs = []
for i, w_id in enumerate(scenarios.index.unique()):
    rt_obj = (demand_res['dRT%s' % w_id] * ushed).sum() - (supply_res['dRT%s' % w_id] * supply['offer ($/MW)']).sum()
    total_obj = da_obj + rt_obj
    objs.append([w_id, total_obj])
obj = pd.DataFrame(objs, columns=['scenario', 'obj']).set_index('scenario')
obj.head()

,obj
scenario,
100,1.132908e+08
101,1.133339e+08
102,1.133408e+08
103,1.132250e+08
104,1.132966e+08


In [20]:
assert M.primalObjValue() - obj['obj'].mean() < 1e-6
obj.to_csv('output/test_objective.csv')

In [21]:
print(log.getvalue())

MOSEK warning 705: #1 (nearly) zero elements are specified in sparse row ''(3) of matrix 'A'.
MOSEK warning 705: #1 (nearly) zero elements are specified in sparse row ''(4) of matrix 'A'.
MOSEK warning 705: #1 (nearly) zero elements are specified in sparse row ''(5) of matrix 'A'.
MOSEK warning 705: #1 (nearly) zero elements are specified in sparse row ''(3433) of matrix 'A'.
MOSEK warning 705: #1 (nearly) zero elements are specified in sparse row ''(3434) of matrix 'A'.
MOSEK warning 705: #1 (nearly) zero elements are specified in sparse row ''(3435) of matrix 'A'.
MOSEK warning 705: #1 (nearly) zero elements are specified in sparse row ''(6862) of matrix 'A'.
MOSEK warning 705: #1 (nearly) zero elements are specified in sparse row ''(6863) of matrix 'A'.
MOSEK warning 705: #1 (nearly) zero elements are specified in sparse row ''(6864) of matrix 'A'.
MOSEK warning 705: #1 (nearly) zero elements are specified in sparse row ''(10291) of matrix 'A'.
Warning number 705 is disabled.
Proble